In [ ]:
#!git clone "https://github.com/SriRamGovardhanam/wastedata-Mask_RCNN-multiple-classes.git"

# !pip install -r requirements.txt
#%cd ./wastedata-Mask_RCNN-multiple-classes/main/Mask_RCNN
#!pip install -e .

In [1]:
import skimage
print(skimage.__version__)

0.16.2


<b>usefull links </b>
https://towardsdatascience.com/computer-vision-instance-segmentation-with-mask-r-cnn-7983502fcad1
https://towardsdatascience.com/mask-rcnn-implementation-on-a-custom-dataset-fd9a878123d4

<b> Please restart </b>

In [1]:
#%cd /content/drive/MyDrive/PAVE DOUGH DEVELOPMENT (USMAN)/Instance Segmentation Pave
import os
os.listdir()

['.ipynb_checkpoints',
 'datasets-20220628T102320Z-004.zip',
 'all class_orig_data.ipynb',
 'wastedata-Mask_RCNN-multiple-classes',
 'datasets-20220628T102320Z-002.zip',
 'datasets',
 'datasets-20220628T102320Z-001.zip',
 'datasets-20220628T102320Z-003.zip']

In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

2022-06-28 16:05:42.910036: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2022-06-28 16:05:42.934620: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3000000000 Hz
2022-06-28 16:05:42.936739: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55eb9b9e6a60 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-06-28 16:05:42.936755: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-06-28 16:05:42.938880: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-06-28 16:05:43.318499: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55eb9b9d28e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-06-28 16:05:43.318549: I tensorflow/compil

'/device:GPU:0'

In [3]:
from mrcnn.config import Config
import os
import numpy as np

class CustomConfig(Config):
    """Configuration for training on the dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "object"
    print( 'hello' )
    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 2
    IMAGE_RESIZE_MODE = "square"
    IMAGE_SHAPE = np.array([1024, 1024  ,3])
    GPU_COUNT = 1
    # Number of training steps per epoch
    STEPS_PER_EPOCH = 300
    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.5

    LEARNING_RATE = 0.01
    DEFAULT_LOGS_DIR="/content/drive/MyDrive/PAVE DOUGH DEVELOPMENT (USMAN)/Instance Segmentation Pave/logs"
    ROOT_DIR = os.path.abspath("./wastedata-Mask_RCNN-multiple-classes/main/Mask_RCNN/")
    Config.WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
    # CUSTOM_DIR = os.path.join(ROOT_DIR, "./dataset/")
    DATA_DIR = './datasets/fetch_v1/split_dataset_objects_wise'
    colorCodePath = './datasets/old/dataset_v1/labels_mapping.xlsx'
    # Directory to save logs and model checkpoints
    DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")
    IMAGE_MAX_DIM=1024
    MAX_GT_INSTANCES=25
 
    #train_subset, val_subset,test_subset = 'training', 'validation','testing'
    train_subset, val_subset = 'training', 'validation'
    STEPS_PER_EPOCH = int( len(  os.listdir( os.path.join(DATA_DIR, train_subset) ) ) / IMAGES_PER_GPU )-2
    VALIDATION_STEPS = int( len(  os.listdir( os.path.join(DATA_DIR, val_subset) ) ) / IMAGES_PER_GPU )-2
    #TESTING_STEPS = int( len(  os.listdir( os.path.join(DATA_DIR, test_subset) ) ) / IMAGES_PER_GPU )-2
    # classes = ['crack']
     # Number of classes (including background)
    NUM_CLASSES = 9  # Background + (other labels)
    classes = ['conc-slb', 'conc-mrk','conc-crk','conc-msk-long-ltrt', 'conc-spl', 'conc-pat', 'conc-cut','bridge']
    # labels_filename = 'labels.json'
    json_input_folder_path = './datasets/fetch_v1/split_dataset_objects_wise/jsons'


hello


In [4]:
#%cd /content/drive/MyDrive/PAVE DOUGH DEVELOPMENT (USMAN)/Instance Segmentation Pave

import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
import pandas as pd
from mrcnn.visualize import display_instances
import matplotlib.pyplot as plt
#from config import CustomConfig 
# Root directory of the project

# Import Mask RCNN
 # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils
# Path to trained weights file
# sys.path.append(ROOT_DIR) 
#from dataset_preparation import CustomDataset


configs = CustomConfig()
(configs.IMAGE_SHAPE)
print(configs.display())


Using TensorFlow backend.



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DATA_DIR                       ./datasets/fetch_v1/split_dataset_objects_wise
DEFAULT_LOGS_DIR               /home/yun-lab/usman projects/PAVE dOUGH PROJECT/INSTANCE SEGMENTATION MODELING/MASKRCNN/Training1/wastedata-Mask_RCNN-multiple-classes/main/Mask_RCNN/logs
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                21
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MO

In [14]:
import tensorflow as tf
import keras

In [ ]:
# instances_path = '/content/drive/MyDrive/Mask_RCNN/mask_rcnn_instances'

# if not os.path.exists(instances_path):
#   os.makedirs(instances_path)

# !python /content/maskrccn_parse_json.py

<b> Data Generator to Parse the setup the data streams </b>

# Data generators

In [ ]:
configs.classes

In [5]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2, glob
from mrcnn.visualize import display_instances
import matplotlib.pyplot as plt

import os,glob
import shutil
import random
import pickle
#from config import CustomConfig 
# Root directory of the project
# ROOT_DIR = os.path.abspath("./wastedata-Mask_RCNN-multiple-classes/main/Mask_RCNN/")
# Import Mask RCNN
# sys.path.append(ROOT_DIR)  # To find local version of the library
# from mrcnn.config import Config
# from mrcnn import model as utils
from mrcnn import model as modellib, utils
class CustomDataset1(utils.Dataset):

    def load_custom(self, dataset_dir, subset):
        """Load a subset of the Horse-Man dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
        self.configs = CustomConfig()
        self.name_dict = {}

        df = pd.read_excel( self.configs.colorCodePath )
        colorDict = dict(zip(df.Label, df.Color))

        classes = self.configs.classes
        print(classes, configs.NUM_CLASSES)
        count=1
        for clas in classes:
          self.add_class("object", count, clas)
          self.name_dict[clas] = count
          count += 1

        # Train or validation dataset?
        
        assert subset in [self.configs.train_subset, self.configs.val_subset]
        dataset_dir = os.path.join(dataset_dir, subset)

        # annotations1 = json.load(open(os.path.join(dataset_dir, self.configs.labels_filename )))
        image_files_list = glob.glob(os.path.join(dataset_dir,'*'))
        for index1, path_iter in enumerate(image_files_list):
          
            num_ids = []
            filename = os.path.split(path_iter)[-1]
            json_filepath = os.path.join(self.configs.json_input_folder_path, filename.replace('.jpg', '_mask.json'))
            file_object = open(json_filepath)
            data = json.load(file_object)
            data_2 =  data['lsfs']['99']['objects']

            json_info, num_ids = self.parse_json(data_2)

            # file_iter = os.path.split(path_iter)[-1]
            imagepath = path_iter
            
            image = skimage.io.imread(imagepath)
            height, width = image.shape[:2]

            self.add_image(
                "object",  ## for a single class just add the name here
                image_id=filename,  # use file name as a unique image id
                path= imagepath,
                width=width, height=height,
                instance_data= json_info,
                num_ids=num_ids
                )

    def parse_json(self, data_2):
    
        json_info, num_ids =[], []
        for i in range(len(data_2)):
            curr = {}
        # print(i)
            if data_2[i]['deleted']==True:
              continue
            
            else:
              if data_2[i]['label'] in configs.classes:   #== 'conc-slb':
                curr['coordinates'] =data_2[i]['coordinates']
                curr['thickness'] = int(data_2[i]['thickness'])
                curr['type'] = data_2[i]['type']

                curr['label']   = data_2[i]['label']
                num_ids.append( self.name_dict[curr['label']] )
                json_info.append(curr)
                  
                  
        return json_info, num_ids

    def draw_poly(self, img, cordinates, colorCode):
        pts = np.array(cordinates, np.int32).reshape((-1, 2))
        img=cv2.fillPoly(img, pts=[pts],  color=colorCode )

        return img

    def draw_instance(self, mask, curr):
          
          # print(curr['label'], self.configs.classes)
          if curr['label'] in self.configs.classes:
              # print(curr)
              if curr['type'] == 'Polygon':
                  mask = self.draw_poly(mask, curr['coordinates'], 1)
          
              else:
                  for k in range(len(curr['coordinates'])-1):   
                    # print(k, curr['coordinates'][k], curr['coordinates'][k+1])
                    mask = cv2.line(mask, pt1=tuple(curr['coordinates'][k]), pt2=tuple(curr['coordinates'][k+1]), color=(1,1), thickness= curr['thickness'] )

          return mask
    
    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a Horse/Man dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        # print('info', info)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["instance_data"])],dtype=np.uint8)
        
        for i, p in enumerate(info["instance_data"]):
          # Get indexes of pixels inside the polygon and set them to 1
          single_mask = np.zeros( (info["height"], info["width"]), dtype=np.uint8)
          single_mask = self.draw_instance(single_mask, p)
          # rr, cc = skimage.draw.polygon(np.array(p['all_points_y'])-1, np.array(p['all_points_x'])-1)
          # rgb = np.dstack((single_mask,single_mask,single_mask))
          # rgb_resized = cv2.resize(rgb.astype('uint8'),(1024,1024))

          mask[:, :, i] = single_mask
          
        # from google.colab.patches import cv2_imshow
        # print(img.shape)
        # cv2_imshow(img)
        # break

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        # Map class names to class IDs.
        num_ids = np.array(num_ids, dtype=np.int32)
        # print(mask.shape, num_ids.shape)
        # print(image_info )
        return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            # print(info['path'])
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [6]:

dataset_train = CustomDataset1()
dataset_train.load_custom(configs.DATA_DIR, "training")
# print(configs.DATA_DIR)
dataset_train.prepare()

mask = dataset_train.load_mask(1)
print(mask[0].shape, mask[1])

# # file_name = os.path.split(dataset_train.image_reference(k) )[-1]

# final_mask = np.zeros( ( mask[0].shape[:2] + (3,) ) )
# colorcodes = []
# for i in range( mask[0].shape[2] ):

#   instance1 = mask[0][:,:, i]
#   colorcodes.append(tuple( np.random.randint(256, size=3) ))
#   final_mask[instance1==1] = colorcodes[-1]
# from google.colab.patches import cv2_imshow
# cv2_imshow(final_mask)

['conc-slb', 'conc-mrk', 'conc-crk', 'conc-msk-long-ltrt', 'conc-spl', 'conc-pat', 'conc-cut', 'bridge'] 9
(3381, 2150, 6) [4 1 1 1 2 2]


In [ ]:
np.unique(mask[0][:,:,0])

In [ ]:
plt.imshow(mask[0][:,:,2])

In [ ]:
#for _id_iter in (range(len(dataset_train.image_ids))):
image_id = 10
print(image_id)
image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_train, configs, image_id, use_mini_mask=False)
print(gt_mask.shape,image.shape)
print(gt_class_id)
  # info = dataset_val.image_info[image_id]
  # print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id,dataset_val.image_reference(image_id)))
  # # Run object detection
  # results = model.detect([image], verbose=1)
  # # Display results
  # x = get_ax(1)
  # r = results[0]
  # visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], dataset_val.class_names, r['scores'], ax=x, title="Predictions")
  # plt.savefig(os.path.join(outputpath_results,str(_id_iter)+'.jpg'), dpi='figure')
  # log("gt_class_id", gt_class_id)
  # log("gt_bbox", gt_bbox)
  # log("gt_mask", gt_mask)

In [ ]:
plt.imshow(gt_mask[:,:,0])

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(image)

In [ ]:
# base_dir = './masks'
# #finding countours and bounding boxes
# def find_contours(img):
    
#     img = np.where(img==(54,  67, 244),255,0)
#     result = np.array(img).astype('uint8')
    
#     contours = cv2.findContours(np.array(img[:,:,0]).astype('uint8'), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     contours = contours[0] if len(contours) == 2 else contours[1]
#     for cntr in contours[:1]:
#         # cntr1 = cntr
#         # exit()
#         x,y,w,h = cv2.boundingRect(cntr)
#         cv2.rectangle(result, (x, y), (x+w, y+h), (255, 255, 255), 32)
#         print("x,y,w,h:",x,y,w,h)

#     # print(result)
#     return result
# # plt.imshow(result)

# # Validation dataset
# dataset_train = CustomDataset()
# dataset_train.load_custom(configs.DATA_DIR, "training")
# # print(configs.DATA_DIR)
# dataset_train.prepare()

# for k in range(300):

#   mask = dataset_train.load_mask(k)[0]
#   file_name = os.path.split(dataset_train.image_reference(k) )[-1]

#   final_mask = np.zeros( ( mask.shape[:2] + (3,) ) )
#   colorcodes = []
#   for i in range( mask.shape[2] ):

#     instance1 = mask[:,:, i]
#     colorcodes.append(tuple( np.random.randint(256, size=3) ))
#     final_mask[instance1==1] = colorcodes[-1]

#     cv2.imwrite(os.path.join(base_dir, f'mask_{file_name}' ), final_mask )
#     img = cv2.imread(dataset_val.image_reference(i))

#     contr_filename = file_name.replace('.jpg', '.png')
#     print(contr_filename)
#     # /content/final data/masks/201021_155645975_Camera_7.png
#     contr_mask = cv2.imread(f'/content/final data/masks/{contr_filename}')
#     result = find_contours( contr_mask ) 
#     cv2.imwrite( os.path.join(base_dir, f'{file_name}' ), img )
#     # print(result.shape)
#     a = cv2.imwrite(os.path.join(base_dir, f'contour_{file_name}' ), result )
#     print(a)
#     # plt.imsave(os.path.join(base_dir, f'contour_{file_name}' ), result)
#   # print(k)

# Model Training

In [7]:
def train(model):
    """Train the model."""
    # Training dataset.
    dataset_train = CustomDataset1()
    dataset_train.load_custom(configs.DATA_DIR, "training")
    # print(configs.DATA_DIR)
    dataset_train.prepare()

    # Validation dataset
    dataset_val = CustomDataset1()
    dataset_val.load_custom(configs.DATA_DIR, "validation")
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    model.train(dataset_train,dataset_val,
                learning_rate=configs.LEARNING_RATE,
                
                epochs=25,
                layers='3+')

In [ ]:

import warnings
import pandas as pd
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
warnings.simplefilter(action='ignore', category=FutureWarning)

# config = CustomConfig()
model = modellib.MaskRCNN(mode="training", config=configs,
                                  model_dir=configs.DEFAULT_LOGS_DIR)

weights_path = configs.WEIGHTS_PATH
# weights_path = "./wastedata-Mask_RCNN-multiple-classes/main/Mask_RCNN/logs/object20220412T0723/mask_rcnn_object_0014.h5"
        # Download weights file
if not os.path.exists(weights_path):
  utils.download_trained_weights(weights_path)


model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])
train(model)






Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead







2022-06-28 16:07:11.289410: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: NVIDIA GeForce RTX 3090 major: 8 minor: 6 memoryClockRate(GHz): 1.8
pciBusID: 0000:1b:00.0
2022-06-28 16:07:11.289498: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2022-06-28 16:07:11.289525: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2022-06-28 16:07:11.289548: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10.0
2022-06-28 16:07:11.289570: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10.0
2022-06-28 16:07:11.289591: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusolver.so.10.0
2022-06-28 16:07:11.289613: I tensorflow/stream_execut

['conc-slb', 'conc-mrk', 'conc-crk', 'conc-msk-long-ltrt', 'conc-spl', 'conc-pat', 'conc-cut', 'bridge'] 9
['conc-slb', 'conc-mrk', 'conc-crk', 'conc-msk-long-ltrt', 'conc-spl', 'conc-pat', 'conc-cut', 'bridge'] 9
Training network heads

Starting at epoch 0. LR=0.01

Checkpoint Path: /home/yun-lab/usman projects/PAVE dOUGH PROJECT/INSTANCE SEGMENTATION MODELING/MASKRCNN/Training1/wastedata-Mask_RCNN-multiple-classes/main/Mask_RCNN/logs/object20220628T1611/mask_rcnn_object_{epoch:04d}.h5
Selecting layers to train
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
res3a_branch2b         (Conv2D)
bn3a_branch2b          (BatchNorm)
res3a_branch2c         (Conv2D)
res3a_branch1          (Conv2D)
bn3a_branch2c          (BatchNorm)
bn3a_branch1           (BatchNorm)
res3b_branch2a         (Conv2D)
bn3b_branch2a          (BatchNorm)
res3b_branch2b         (Conv2D)
bn3b_branch2b          (BatchNorm)
res3b_branch2c         (Conv2D)
bn3b_branch2c          (BatchNorm)
res3c_branch2

/home/yun-lab/anaconda3/envs/maskcrcnn_fastapi/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/yun-lab/anaconda3/envs/maskcrcnn_fastapi/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/yun-lab/anaconda3/envs/maskcrcnn_fastapi/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor





/home/yun-lab/anaconda3/envs/maskcrcnn_fastapi/lib/python3.7/site-packages/keras/engine/training.py:1987: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/25


2022-06-28 16:13:39.633425: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2022-06-28 16:14:49.764764: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2022-06-28 16:29:17.399896: W tensorflow/stream_executor/cuda/redzone_allocator.cc:312] Not found: ./bin/ptxas not found
Relying on driver to perform ptx compilation. This message will be only logged once.


# **Testing**

In [ ]:
# import os
# import sys
# import random
# import math
# import re
# import time
# import numpy as np
# import tensorflow as tf
# import matplotlib
# import matplotlib.pyplot as plt
# import matplotlib.patches as patches
# import matplotlib.image as mpimg
# # Root directory of the project
# #ROOT_DIR = os.path.abspath("/")

# # Import Mask RCNN
# sys.path.append(configs.ROOT_DIR)  # To find local version of the library
# from mrcnn import utils
# from mrcnn import visualize
# from mrcnn.visualize import display_images
# import mrcnn.model as modellib
# from mrcnn.model import log
# %matplotlib inline
# # Directory to save logs and trained model
# MODEL_DIR = os.path.join(configs.ROOT_DIR, "logs")
# # Path to Ballon trained weights
# # You can download this file from the Releases page
# # https://github.com/matterport/Mask_RCNN/releases
# WEIGHTS_PATH = "./wastedata-Mask_RCNN-multiple-classes/main/Mask_RCNN/logs/object20220519T2226/mask_rcnn_object_0006.h5"  # TODO: update this path

In [ ]:
configs = CustomConfig()
CUSTOM_DIR = os.path.join(configs.ROOT_DIR, configs.DATA_DIR)
class InferenceConfig(configs.__class__):
  # Run detection on one image at a time
  GPU_COUNT = 1
  IMAGES_PER_GPU = 1
  #STEPS_PER_EPOCHS=10
  DETECTION_MIN_CONFIDENCE = 0.6

configs_infer = InferenceConfig()
configs_infer.display()

In [ ]:
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0
# Inspect the model in training or inference modes values: 'inference' or 'training'
TEST_MODE = "inference"
def get_ax(rows=1, cols=1, size=16):
  # ""Return a Matplotlib Axes array to be used in all visualizations in the notebook. Provide a central point to control graph sizes. Adjust the size attribute to control how big to render images"""
  _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
  return ax

# Load validation dataset

dataset_val = CustomDataset1()
dataset_val.load_custom(configs.DATA_DIR, "validation")
# print(configs.DATA_DIR)
dataset_val.prepare()

print("Images: {}\nClasses: {}".format(len(dataset_val.image_ids), dataset_val.class_names))

## **Loading model**

In [ ]:
import tensorflow as tf
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0
# Inspect the model in training or inference modes values: 'inference' or 'training'
TEST_MODE = "inference"
with tf.device(DEVICE):
  model = modellib.MaskRCNN(mode="inference", model_dir=configs.DEFAULT_LOGS_DIR, config=configs_infer)

#weights_path = WEIGHTS_PATH
weights_path = "./wastedata-Mask_RCNN-multiple-classes/main/Mask_RCNN/logs/object20220619T1446/mask_rcnn_object_0022.h5"
# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

In [ ]:
# from mrcnn import utils
# from mrcnn import visualize
# #from mrcnn.visualize import display_images
# import mrcnn.model as modellib
# from mrcnn.model import log

# output_path = '/content/drive/MyDrive/PAVE DOUGH DEVELOPMENT (USMAN)/Instance Segmentation Pave'
# outputpath_results = os.path.join(output_path,'slb-mark-crck_results')
# if not os.path.exists(outputpath_results):
#   os.makedirs(outputpath_results)

# def get_ax(rows=1, cols=1, size=16):
#   # ""Return a Matplotlib Axes array to be used in all visualizations in the notebook. Provide a central point to control graph sizes. Adjust the size attribute to control how big to render images"""
#   _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
#   return ax

# for _id_iter in (range(len(dataset_val.image_ids))):
#   image_id = _id_iter
#   print(image_id)
#   image, image_meta, gt_class_id, gt_bbox, gt_mask =\
#     modellib.load_image_gt(dataset_val, configs, image_id, use_mini_mask=False)

#   info = dataset_val.image_info[image_id]
#   print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id,dataset_val.image_reference(image_id)))
#   # Run object detection
#   results = model.detect([image], verbose=1)
#   # Display results
#   x = get_ax(1)
#   r = results[0]
#   visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], dataset_val.class_names, r['scores'], ax=x, title="Predictions")
#   plt.savefig(os.path.join(outputpath_results,str(_id_iter)+'.jpg'), dpi='figure')
#   log("gt_class_id", gt_class_id)
#   log("gt_bbox", gt_bbox)
#   log("gt_mask", gt_mask)

## **Saving model prediction 3in1**

In [ ]:
import os
import sys
import random
import itertools
import colorsys

import numpy as np
from skimage.measure import find_contours
import matplotlib.pyplot as plt
from matplotlib import patches,  lines
from matplotlib.patches import Polygon
import IPython.display
def random_colors(N, bright=True):
    """
    Generate random colors.
    To get visually distinct colors, generate them in HSV space then
    convert to RGB.
    """
    brightness = 1.0 if bright else 0.7
    hsv = [(i / N, 1, brightness) for i in range(N)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    random.shuffle(colors)
    return colors

def apply_mask(image, mask, color, alpha=0.5):
    """Apply the given mask to the image.
    """
    for c in range(3):
        image[:, :, c] = np.where(mask == 1,
                                  image[:, :, c] *
                                  (1 - alpha) + alpha * color[c] * 255,
                                  image[:, :, c])
    return image


def display_instances(image, boxes, masks, class_ids, class_names,
                      scores=None, title="",
                      figsize=(16, 16), ax=None,
                      show_mask=True, show_bbox=True,
                      colors=None, captions=None):
    """
    boxes: [num_instance, (y1, x1, y2, x2, class_id)] in image coordinates.
    masks: [height, width, num_instances]
    class_ids: [num_instances]
    class_names: list of class names of the dataset
    scores: (optional) confidence scores for each box
    title: (optional) Figure title
    show_mask, show_bbox: To show masks and bounding boxes or not
    figsize: (optional) the size of the image
    colors: (optional) An array or colors to use with each object
    captions: (optional) A list of strings to use as captions for each object
    """
    # Number of instances
    N = boxes.shape[0]
    if not N:
        print("\n*** No instances to display *** \n")
    else:
        assert boxes.shape[0] == masks.shape[-1] == class_ids.shape[0]

    # If no axis is passed, create one and automatically call show()
    auto_show = False
    if not ax:
        _, ax = plt.subplots(1, figsize=figsize)
        auto_show = True

    # Generate random colors
    colors = colors or random_colors(N)

    # Show area outside image boundaries.
    height, width = image.shape[:2]
    # ax.set_ylim(height + 10, -10)
    # ax.set_xlim(-10, width + 10)
    # ax.axis('off')
    # ax.set_title(title)

    masked_image = image.astype(np.uint32).copy()
    
    for i in range(N):
        color = colors[i]

        # Bounding box
        if not np.any(boxes[i]):
            # Skip this instance. Has no bbox. Likely lost in image cropping.
            continue
        y1, x1, y2, x2 = boxes[i]
        if show_bbox:
            p = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2,
                                alpha=0.7, linestyle="dashed",
                                edgecolor=color, facecolor='none')
            #ax.add_patch(p)
 
        # Label
        if not captions:
            class_id = class_ids[i]
            score = scores[i] if scores is not None else None
            label = class_names[class_id]
            caption = "{} {:.3f}".format(label, score) if score else label
        else:
            caption = captions[i]

        # Mask
        mask = masks[:, :, i]
        if show_mask:
            masked_image = apply_mask(masked_image, mask, color)
        
        x_loc = int(x1+((x2-x1)/2))
        y_loc = int(y1+((y2-y1)/2))
        text_list = [class_names[class_id],str(int(round(scores[i],2)*100))+'%']
        for counter1 in range(2):
          if (int(width) - int(x_loc)) < 300:
            x_loc = int(width)-300
          if (int(height) - int(y_loc)) < 200:
            y_loc = int(height)-300
          
          masked_image = cv2.putText(masked_image.astype('uint8'), text_list[counter1], (x_loc,y_loc), 
                          cv2.FONT_HERSHEY_SIMPLEX,2, (255,255,255), 2, cv2.LINE_AA)
          y_loc = y_loc+55
          x_loc = x_loc+30
    return masked_image




In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2, os, json
from PIL import ImageColor
import argparse
import pandas as pd
import glob
from PIL import Image
from PIL import ImageDraw
import cv2
import numpy as np
import json
import matplotlib.pyplot as plt
import argparse
import os,glob

def color_code_dict(csv_path):
  
  df=pd.read_excel(csv_path)
  return dict(zip(df.Label, df.Color))
# color_dict

def draw_poly(img, cordinates, colorCode):
  pts = np.array(cordinates, np.int32).reshape((-1, 2))
  img=cv2.fillPoly(img, pts=[pts],  color=colorCode )

  return img

def create_mask(data_2, colorDict,imgsize):
  h,w = imgsize#(3483,2149)
  mask = np.ones((h,w, 3)) * 255
  mask =  mask.astype('uint8')
  for i in range(len(data_2)):
    # print(i)
    if data_2[i]['deleted']==True:
      colors = [255,255,255]
    else:
      colors = colorDict[ data_2[i]['label'] ]
      colors = ImageColor.getcolor('#'+str(colors), "RGB")
      #if label is conc-slb
      # if data_2[i]['label'] == 'conc-slb':
      #   colors = ()
      #print(colors)
      

    
    curr=data_2[i]['coordinates']
    colorCode = [colors[2], colors[1], colors[0]]#,100]

    thickness = int(data_2[i]['thickness'])
    if data_2[i]['type'] == 'Polygon':
      mask = draw_poly(mask, curr, colorCode)

    else:
      for k in range(len(curr)-1):

        mask = cv2.line(mask, pt1=tuple(curr[k]), pt2=tuple(curr[k+1]), color=colorCode, thickness= thickness )
 
  return mask


def overlay_3in1_mask(image,mask):

    output = cv2.addWeighted(mask, 0.6, image, 1, 0)
    im_v = cv2.hconcat([image,output, mask])
    return im_v
def apply_mask(image, mask, color, alpha=0.5):
    """Apply the given mask to the image.
    """
    for c in range(3):
        image[:, :, c] = np.where(mask == True,
                                  image[:, :, c] *
                                  (1 - alpha) + alpha * color[c] * 255,
                                  image[:, :, c])
    return image

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg
sys.path.append(configs.ROOT_DIR)  # To find local version of the library
from mrcnn import utils
#from mrcnn import visualize
#from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

dataset_basepath  = 'datasets/fetch_v1/split_dataset_objects_wise'

json_path=os.path.join(dataset_basepath,'jsons')


colorcode_file_path = r'labels_mapping.xlsx'

color_dict = color_code_dict(colorcode_file_path)



# This is for predicting images which are not present in dataset
output_path = './results/fetch_v1_val_10chk'
outputpath_results = os.path.join(output_path,'all_class_testing')
if not os.path.exists(outputpath_results):
  os.makedirs(outputpath_results)

imgs_name = os.listdir(os.path.join(dataset_basepath,'validation'))#'./datasets/dataset_v2/testing')
print("Total images found in the folder: ",len(imgs_name))
for k in range(len(imgs_name)):
  path_to_new_image = os.path.join(dataset_basepath,'validation',imgs_name[k])
  orig_img = cv2.imread(path_to_new_image)
  print(orig_img.shape)
  image1 = mpimg.imread(path_to_new_image)
  print(image1.shape)
  results1 = model.detect([image1], verbose=1)
  # Display results
  ax1 = get_ax(1)
  r1 = results1[0]
  masked_img = display_instances(image1, r1['rois'], r1['masks'], r1['class_ids'], \
                                dataset_val.class_names, r1['scores'], ax=ax1, title="Predictions1")
  

  #applying ground truth
  imagesize = orig_img.shape[:2]
  file_object = open(os.path.join(json_path, imgs_name[k].split('.')[0]+'_mask.json'))


  data = json.load(file_object)
  data_2 =  data['lsfs']['99']['objects']
  #print(file_path)
  mask = create_mask(data_2, color_dict,imagesize)
  orig_image1 = orig_img.copy()
  colors_list = random_colors(len(list(color_dict.keys())))
  #print(color_dict.keys())

  for index1,key_iter in enumerate(color_dict.keys()):
      color = ImageColor.getcolor('#'+str(color_dict[key_iter]), "RGB")
      #print("inner lopp")
      #print([color[2],color[1],color[0]])
      #mask1 = np.where(mask == [color[2],color[1],color[0]],255,0)
      
      mask1 = np.all(mask.astype('uint8') == [color[2],color[1],color[0]], axis=-1)
      
      orig_image1 = apply_mask(orig_image1, mask1, colors_list[index1], alpha=0.5)
      #orig_image1 = apply_mask(orig_image1, mask1, (color[2],color[1],color[0]), alpha=0.5)


  #print("outside inner loop")
  stakced = np.hstack((orig_image1,orig_img,masked_img))
  plt.imshow(stakced)
  check = cv2.imwrite(os.path.join(outputpath_results,imgs_name[k]),stakced.astype('uint8'))
  #print("check:",check)

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg
sys.path.append(configs.ROOT_DIR)  # To find local version of the library
from mrcnn import utils
#from mrcnn import visualize
#from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

colorcode_file_path = r'labels_mapping.xlsx'
json_path=r'datasets/dataset_v2/jsons'
color_dict = color_code_dict(colorcode_file_path)



# This is for predicting images which are not present in dataset
output_path = './results/dataset_v2'
outputpath_results = os.path.join(output_path,'all_class_testing')
if not os.path.exists(outputpath_results):
  os.makedirs(outputpath_results)

imgs_name = os.listdir('./datasets/dataset_v2/testing')
print("Total images found in the folder: ",len(imgs_name))
for k in range(len(imgs_name)):
  path_to_new_image = os.path.join(f'./datasets/dataset_v2/testing/{imgs_name[k]}')
  orig_img = cv2.imread(path_to_new_image)
  print(orig_img.shape)
  image1 = mpimg.imread(path_to_new_image)
  print(image1.shape)
  results1 = model.detect([image1], verbose=1)
  # Display results
  ax1 = get_ax(1)
  r1 = results1[0]
  masked_img = display_instances(image1, r1['rois'], r1['masks'], r1['class_ids'], \
                                dataset_val.class_names, r1['scores'], ax=ax1, title="Predictions1")
  print(masked_img.shape)
  break
  #applying ground truth
  imagesize = orig_img.shape[:2]
  file_object = open(os.path.join(json_path, imgs_name[k].split('.')[0]+'_mask.json'))


  data = json.load(file_object)
  data_2 =  data['lsfs']['99']['objects']
  #print(file_path)
  mask = create_mask(data_2, color_dict,imagesize)
  orig_image1 = orig_img.copy()
  colors_list = random_colors(len(list(color_dict.keys())))
  print(color_dict.keys())

  for index1,key_iter in enumerate(color_dict.keys()):
      color = ImageColor.getcolor('#'+str(color_dict[key_iter]), "RGB")
      print("inner lopp")
      #print([color[2],color[1],color[0]])
      #mask1 = np.where(mask == [color[2],color[1],color[0]],255,0)
      
      mask1 = np.all(mask.astype('uint8') == [color[2],color[1],color[0]], axis=-1)
      
      orig_image1 = apply_mask(orig_image1, mask1, colors_list[index1], alpha=0.5)
      #orig_image1 = apply_mask(orig_image1, mask1, (color[2],color[1],color[0]), alpha=0.5)


  print("outside inner loop")
  stakced = np.hstack((orig_image1,orig_img,masked_img))
  plt.imshow(stakced)
  check = cv2.imwrite(os.path.join(outputpath_results,imgs_name[k]),stakced.astype('uint8'))
  print("check:",check)

In [ ]:
os.listdir()

<b> Prediction on any single image </b>

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg

path_to_new_image = '/content/crack_38.jpg'
image1 = mpimg.imread(path_to_new_image)
results1 = model.detect([image1], verbose=1)
ax1 = get_ax(1)
r1 = results1[0]
print(r1['class_ids'])
maskedimg = display_instances(image1, r1['rois'], r1['masks'], r1['class_ids'], \
                            dataset_val.class_names, r1['scores'], ax=ax1, title="Predictions1")

In [ ]:
plt.imshow(masks1[:,:,0])

In [ ]:
len(r1['class_ids'])

In [ ]:
from PIL import ImageColor
import json

model_classes = ['conc-slb', 'conc-mrk','conc-crk','conc-msk-long-ltrt', 'conc-spl', 'conc-pat', 'conc-cut','bridge']

def populate_json(objects_list):
  jsons1 = {
    "version": "v1.0",
    "lsfs":{
        "300": {
            "id": 300,
            "name": "alg",
            "objects": objects_list}
            }
         }
  return jsons1

def populate_objects(objects_list , dict1):
  objects1 = {
      "id": dict1['id'],
      "type": "Polygon",
      "date": "date",
      "isLabeled": 'true',
      "deleted": 'false',
      "class": dict1['class_label'],
      "thickness": "10",
      "hasOutline": True,
      "coordinates": dict1['cords'],
      "doughCode":dict1['dough_code'], #"12-C00",
      "label": dict1['class_label'],#"asph-crk",
      "labelColor": dict1['hexa_color'],#"#f44336",
      "labelRgbColor": {
          "r": dict1['rgb_color'][0],
          "g": dict1['rgb_color'][0],
          "b": dict1['rgb_color'][0]
          },
        }
  objects_list.append(objects1)
  return objects_list

def write_json(jsons1,path1):
  completeName = '/content/crack_38_mask.json'
  with open(path1, "w") as outfile:
    json.dump(jsons1, outfile)

def color_code_dict(csv_path):
  df=pd.read_csv(csv_path,header=None)
  a = zip(df.iloc[:,0].values.tolist(), df.iloc[:,1].values.tolist(),df.iloc[:,2].values.tolist())
  dict1 = {}
  for index1,i in enumerate(list(a)):
    dict1[list(i)[0]] = {'hexa_color':list(i)[1],'dough_code':list(i)[2]}

  return dict1


def get_json(results1,labels_dict,json_path1):
  
  if len(results1['class_ids']) > 0:

    #finding predicted classes
    predicted_classes = [model_classes[i-1] for i in results1['class_ids']]

    masks1 = results1['masks']
    objects_list = []
    #iterating over predicted masks
    for maskIndex_iter in range(masks1.shape[-1]):
      binary_mask = masks1[:,:,maskIndex_iter].astype('uint8') 

      #finding contours
      contours, hierarchy = cv2.findContours(binary_mask, 
                                             cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
      #Converting contours in polygons
      if len(contours[0]) >= 2:
        
        contours = np.reshape(contours[0],(-1,2))
        polygons1 = []
        for cord_iter in contours:
          polygons1.append( [ int(cord_iter[0]) , int(cord_iter[1])  ])
        polygons1.append([polygons1[0][0],polygons1[0][1]])
        
        dict1={}
        dict1['id'] = int(results1['class_ids'][maskIndex_iter]) 
        dict1['class_label'] = predicted_classes[maskIndex_iter]
        dict1['cords'] = polygons1
        dict1['dough_code'] = labels_dict[predicted_classes[maskIndex_iter]]['dough_code']
        dict1['hexa_color'] = labels_dict[predicted_classes[maskIndex_iter]]['hexa_color']
        dict1['rgb_color'] = ImageColor.getcolor(labels_dict[predicted_classes[maskIndex_iter]]['hexa_color'], "RGB")
        print(dict1['class_label'],dict1['id'],dict1['rgb_color'],dict1['dough_code'],dict1['hexa_color'])
        objects_list = populate_objects(objects_list , dict1) 
    
    jsons1 = populate_json(objects_list)
    write_json(jsons1,json_path1)
    
        
csv_path = r'/content/labels_instance.csv'
json_path1 = '/content/crack_38_mask.json'    
labels_dict = color_code_dict(csv_path)
get_json(r1,labels_dict,json_path1)

In [ ]:
_masks1 = r1['masks']
binary_mask = masks1[:,:,0].astype('uint8') 
contours, hierarchy = cv2.findContours(binary_mask, 
                                             cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
contours[0].shape

In [ ]:
a = np.reshape(contours[0],(-1,2))
c = []
for i in a:
  b=[]
  b.append(int(i[0]))
  b.append(int(i[1]))
  c.append(b)
c.append([c[0][0],c[0][1]])
print(c)

In [ ]:
"id": 0,
                    "type": "Polygon",
                    "date": "",
                    "isLabeled": true,
                    "deleted": false,
                    "doughCode": "11-910",
                    "label": "conc-msk-long-ltrt",
                    "labelColor": "#000000",
                    "labelRgbColor": {
                        "r": 0,
                        "g": 0,
                        "b": 0
                    },
                    "thickness": 1,
                    "class": "none",
                    "coordinates": [

In [ ]:
 objects = {
     "id": 75,
     "type": "Polygon",
     "date": "date",
     "isLabeled": 'true',
     "deleted": 'false',
     "class": "",
     "thickness": "1",
     "hasOutline": False,
     "coordinates": c,
     "doughCode": "12-C00",
     "label": "asph-crk",
     "labelColor": "#f44336",
     "labelRgbColor": {
         "r": 244,
         "g": 67,
         "b": 54
         },
        }
obj_list = []
obj_list.append(objects)

In [ ]:
jsons = {
    "version": "v1.0",
    "lsfs":{
        "200": {
            "id": 200,
            "name": "alg",
            "objects": obj_list}
            }
         }

In [ ]:
import json
completeName = '/content/crack_38_mask.json'
with open(completeName, "w") as outfile:
    json.dump(jsons, outfile)

In [ ]:
import json
jsonpath = '/content/crack_11_mask.json'
file_object = open(file_iter)
data = json.load(file_object)
data_2 =  data['lsfs']['99']['objects']

## **Prediction 2in1**

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg
# Root directory of the project
#ROOT_DIR = os.path.abspath("/")

# Import Mask RCNN
sys.path.append(configs.ROOT_DIR)  # To find local version of the library
from mrcnn import utils
#from mrcnn import visualize
#from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log


# This is for predicting images which are not present in dataset
output_path = '/content/drive/MyDrive/PAVE DOUGH DEVELOPMENT (USMAN)/Instance Segmentation Pave/results/dataset_v2'
outputpath_results = os.path.join(output_path,'all_class_validation')
if not os.path.exists(outputpath_results):
  os.makedirs(outputpath_results)

imgs_name = os.listdir('./dataset_v2/validation')
for k in range(len(imgs_name)):
  path_to_new_image = os.path.join(f'./dataset_v2/validation/{imgs_name[k]}')
  orig_img = cv2.imread(path_to_new_image)
  print(orig_img.shape)
  image1 = mpimg.imread(path_to_new_image)
  results1 = model.detect([image1], verbose=1)
  # Display results
  ax1 = get_ax(1)
  r1 = results1[0]
  masked_img = display_instances(image1, r1['rois'], r1['masks'], r1['class_ids'], \
                                dataset_val.class_names, r1['scores'], ax=ax1, title="Predictions1")
  print(masked_img.shape)
  stakced = np.hstack((orig_img,masked_img))
  cv2.imwrite(os.path.join(outputpath_results,imgs_name[k]),stakced.astype('uint8'))

<b>populating json </b>

In [ ]:
# # from imantics import Polygons, Mask
# import cv2, glob, os

# img_paths = glob.glob( os.path.join('/content/final data/original images', '*.jpg') )
# mask_paths = glob.glob( os.path.join('/content/final data/masks', '*.png') )

# split = int( len(mask_paths)*0.7 )
# train_mask_paths, validation_mask_paths = mask_paths[:split], mask_paths[split: ]
# # train_image_paths, validation_image_paths = img_paths[:100], img_paths[100: 125]

# def get_masks(path):

#     image = cv2.imread(path)
#     crack_color, bg_color = (54,  67, 244), (255, 255, 255)
#     # legal_colors = [patch_color, bg_color]
#     img = np.where(image==(54,  67, 244),255,0)
#     if len(np.unique(img)) == 1:
#       return {}

#     #finding countours and bounding boxes
#     result = np.array(img).astype('uint8')
#     contours = cv2.findContours(np.array(img[:,:,0]).astype('uint8'), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     contours = contours[0] if len(contours) == 2 else contours[1]
#     count = 0
#     curr_dict = { 'fileref': '', 'size': '', 'filename': os.path.split(path)[-1].replace('.png', '.jpg'), 'base64_img_data': '',
#                 'file_attributes': '', 'regions': {}}
#     for cntr in contours:
#       x_points = [ int(cnt[0][0]) for cnt in cntr]
#       y_points = [ int(cnt[0][1]) for cnt in cntr]
#       temp = {'shape_attributes':{ 'name': 'Polygon', 'all_points_x': x_points,\
#                                    'all_points_y': y_points }, "region_attributes":{"label":"crack"} }
#       curr_dict['regions'][f'{count}'] = temp
#       count += 1
#       # print(contours)
#       # x,y,w,h = cv2.boundingRect(cntr)
#       # cv2.rectangle(result, (x, y), (x+w, y+h), (255, 255, 255), 32)
#       # print("x,y,w,h:",x,y,w,h)


#     # illegal_colors = []
#     # for k in 

    
#     # unique_colors = np.unique(image.reshape(-1, image.shape[2]), axis=0)
#     # illegal_colors = [ list(k) for k in list(np.unique(image.reshape(-1, image.shape[2]), axis=0))\
#                         # if not tuple(k) in legal_colors]

#     # for color in illegal_colors:
#       # image[np.all(image == tuple(color), axis=-1)] = bg_color
#     # cords=  np.where(image==patch_color)
#     # x_coordinates, y_coordinates =  cords[0].tolist(), cords[1].tolist()
    
#     # poly_points = []
#     # labels_dict = {}
#     # img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#     # curr_dict={}

#     # region_dict = { 'name': 'line', 'all_points_x': x_coordinates, 'all_points_y': y_coordinates}
#     # curr_dict = { 'fileref': '', 'size': '', 'filename': os.path.split(img_path)[-1], 'base64_img_data': '',
#                 # 'file_attributes': '', 'regions': {}, 'region_attributes': 'Patch'}


#     # labels_dict[os.path.split(img_path)[-1]] = curr_dict
#     # polygons = Mask(img).polygons()
#     # poly_points.append((polygons.points)[0])

#     return curr_dict

In [ ]:
# import os, json

# print( len( os.listdir('/content/drive/MyDrive/Mask_RCNN/crack_data/training') ) )
# # data = json.loads('/content/drive/MyDrive/Mask_RCNN/crack_data/training/labels.json')

# json_file_path = "/content/drive/MyDrive/Mask_RCNN/crack_data/training/labels.json"

# with open(json_file_path, 'r') as j:
#      contents = json.loads(j.read())

# len( contents.keys() )

In [ ]:
# # cords=  np.where(image==patch_color)
# # x_coordinates, y_coordinates =  cords[0].tolist(), cords[1].tolist()
# import json

# base_dir = '/content/drive/MyDrive/Mask_RCNN/crack_data'
# if not os.path.exists(base_dir):
#   os.makedirs(base_dir)

# if not os.path.exists('/content/drive/MyDrive/Mask_RCNN/crack_data/training'):
#   os.makedirs( '/content/drive/MyDrive/Mask_RCNN/crack_data/training' )


# labels_dict = {}

# for mask_path in train_mask_paths:

#   result_dict= get_masks(mask_path)
#   if len(result_dict) == 0:
#     continue

#   labels_dict[os.path.split(mask_path)[-1]] = result_dict
#   # break
#   extensionles_path = os.path.split(mask_path)[-1].split('.')[0]
#   image = cv2.imread( os.path.join('/content/final data/original images/', f'{extensionles_path}.jpg' ))
#   # print(image.shape)
#   cv2.imwrite(os.path.join('/content/drive/MyDrive/Mask_RCNN/crack_data/training', f'{extensionles_path}.jpg'), image )

# label_path = '/content/drive/MyDrive/Mask_RCNN/crack_data/training/labels.json'
# with open(label_path, 'w') as json_file:
#     json.dump(labels_dict, json_file)

In [ ]:
# # cords=  np.where(image==patch_color)
# # x_coordinates, y_coordinates =  cords[0].tolist(), cords[1].tolist()
# import json

# # base_dir = '/content/drive/MyDrive/Mask_RCNN/patch_data'
# # if not os.path.exists(base_dir):
#   # os.makedirs(base_dir)

# if not os.path.exists('/content/drive/MyDrive/Mask_RCNN/crack_data/validation'):
#   os.makedirs( '/content/drive/MyDrive/Mask_RCNN/crack_data/validation' )


# labels_dict = {}

# for mask_path in validation_mask_paths:

#   result_dict= get_masks(mask_path)
#   if len(result_dict) == 0:
#     continue

#   labels_dict[os.path.split(mask_path)[-1]] = result_dict

#   extensionles_path = os.path.split(mask_path)[-1].split('.')[0]
#   image = cv2.imread( os.path.join('/content/final data/original images/', f'{extensionles_path}.jpg' ))
#   print(image.shape)
#   cv2.imwrite(os.path.join('/content/drive/MyDrive/Mask_RCNN/crack_data/validation', f'{extensionles_path}.jpg'), image )

# label_path = '/content/drive/MyDrive/Mask_RCNN/crack_data/validation/labels.json'
# with open(label_path, 'w') as json_file:
#     json.dump(labels_dict, json_file)

In [ ]:
# import cv2
# import numpy as np
# import matplotlib.pyplot as plt

# # read image
# img = cv2.imread('/content/final data/masks/201021_170008099_Camera_7.png')
# print(img.shape)
# #only getting cracks mask

# img = np.where(img==(54,  67, 244),255,0)

# #finding countours and bounding boxes
# result = np.array(img).astype('uint8')
# contours = cv2.findContours(np.array(img[:,:,0]).astype('uint8'), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# contours = contours[0] if len(contours) == 2 else contours[1]
# for cntr in contours:
#     cntr1 = cntr
#     # exit()
#     x,y,w,h = cv2.boundingRect(cntr)
#     cv2.rectangle(result, (x, y), (x+w, y+h), (255, 255, 255), 32)
#     print("x,y,w,h:",x,y,w,h)
# plt.imshow(result)